<h1>Image Exporter for Google Earth Engine</h1>

The code below is the first version of the image exporter. It takes a KML file that's imported from google earth pro, and transforms the polygon in the file into an image that is then exported to google drive

<h2>Imports</h2>

In [ ]:
import ee
import geemap
import geopandas as gpd
from shapely.ops import transform

<h2>Authenticating and Initializing Google Earth Engine</h2>

In [ ]:
# Initialize
try:
    ee.Initialize(project='golf-mapper-463720')
    print("Earth Engine initialized successfully.")
except Exception as e:
    print(f"Initialization failed: {e}")
    ee.Authenticate()
    ee.Initialize(project='golf-mapper-463720')

<h2>Data Cleaning, File Conversion</h2>
By default, the KML file from Earth Pro has a Z-axis, which is unable to be interpreted by Earth Engine. The code below removes the Z-axis component from the file. After the removal, the code converts the KML file into an Earth Engine Object. Lastly, the geometry is converted into.a 'Geometry' type variable, which allows it to be passed to the image clipping and export functions. 

NOTE: the image can use the 'geometry variable', but later on when we export to drive it needs to be a 'Geometry' type variable, which is called 'my_region' in this cell

In [ ]:
# Read your KML
#gdf = geopandas data frame
#gdf_to_ee() is a built-in earthengine function
gdf = gpd.read_file('/Users/carrollcj/proj/golfmapper/kml_files/test3.kml')

# Function to remove Z coordinates
def remove_z_dimension(geom):
    if geom.has_z:
        return transform(lambda x, y, z=None: (x, y), geom)
    return geom

# Apply to all geometries
gdf['geometry'] = gdf.geometry.apply(remove_z_dimension)

# Verify it worked
print("After Z removal:")
print("Geometry types:", gdf.geometry.type.value_counts())
print("Sample geometry:", gdf.geometry.iloc[0])

# Now try the Earth Engine conversion
geometry = geemap.gdf_to_ee(gdf)
print("Success! Converted to Earth Engine geometry")
print(f"GEOMETRY TYPE: {type(geometry)}")
print('===============')

print('Converting geometry type...')
my_region = geometry.geometry()
print(f"CONVERTED REGION TYPE: {type(my_region)} ✅")
print(f"CONVERTED REGION INFO: {my_region.getInfo()}")
print('===============')

#We will pass the 'my_region' variable to the exporter function later on

After Z removal:
Geometry types: Polygon    1
Name: count, dtype: int64
Sample geometry: POLYGON ((-95.37972929310104 37.65883757523466, -95.35579120123668 37.65832987171771, -95.35416643387549 37.66965512912511, -95.3815263901972 37.66997271533956, -95.37972929310104 37.65883757523466)) 😑
Success! Converted to Earth Engine geometry
GEOMETRY TYPE: <class 'ee.featurecollection.FeatureCollection'>
Converting geometry type...
CONVERTED REGION TYPE: <class 'ee.geometry.Geometry'> ✅
CONVERTED REGION INFO: {'type': 'Polygon', 'coordinates': [[[-95.37972929310104, 37.65883757523466], [-95.35579120123668, 37.65832987171771], [-95.35416643387549, 37.66965512912511], [-95.3815263901972, 37.66997271533956], [-95.37972929310104, 37.65883757523466]]]}


<h2>Clipping the Satellite Image of the Region</h2>

Using images from the sentinel satellite

In [9]:
start_date = '2023-01-01'
end_date = '2023-12-31'

# Get Sentinel-2 imagery
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()  # Get median composite

# Clip the image to your polygon
clipped_image = sentinel.clip(geometry)

<h2>Exporting the image to Google Drive</h2>

FIXME:
- need to figure out the scale variable
- how to get an actual image not just black and white.


In [12]:
task = ee.batch.Export.image.toDrive(
    image=clipped_image,
    description='clipped_image_test',
    folder='GEE_Export_Test',  # Optional: creates folder in Drive
    fileNamePrefix='test_30scale',
    scale=30,  # Adjust resolution
    region=my_region,
    maxPixels=1e9
)
task.start()

print(f"Task status: {task.status()}")

Task status: {'state': 'READY', 'description': 'clipped_image_test', 'priority': 100, 'creation_timestamp_ms': 1752455619987, 'update_timestamp_ms': 1752455619987, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '7XODL5R3SDF3XMFPTJYZQ5U5', 'name': 'projects/golf-mapper-463720/operations/7XODL5R3SDF3XMFPTJYZQ5U5'}
